In [18]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input , Session

In [19]:

my_region=boto3.session.Session().region_name
print(my_region)

eu-north-1


In [20]:
import boto3

s3 = boto3.resource('s3')

my_region = 'eu-north-1'
bucket_name = 'bankapplicationnew12'
if my_region == 'eu-north-1':
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': my_region}
    )
else:
    s3.create_bucket(Bucket=bucket_name)

print("S3 bucket created successfully")


S3 bucket created successfully


In [21]:
# Download data
try:
    urllib.request.urlretrieve(
        "https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv",
        "bank_clean.csv"
    )
    print("Data download success")
except Exception as e:
    print("Data load error", e)

# Load data
model_data = pd.read_csv('./bank_clean.csv', index_col=0)

Data download success


In [22]:
# Split data into training and testing
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

# Save training and testing data to CSV
train_data.to_csv('train.csv', index=False, header=True)
test_data.to_csv('test.csv', index=False, header=True)

# Inspect the columns in the training data
train_df = pd.read_csv('train.csv')
print("Training data columns:", train_df.columns)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


(28831, 61) (12357, 61)
Training data columns: Index(['age', 'campaign', 'pdays', 'previous', 'no_previous_contact',
       'not_working', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_se

In [23]:
# Replace 'y_yes' with the correct label column name if different
label_column = 'y_yes'

# Function to convert CSV to LIBSVM format
def csv_to_libsvm(input_csv_path, output_libsvm_path, label_column):
    data = pd.read_csv(input_csv_path)
    y = data[label_column]
    X = data.drop(columns=[label_column])
    dump_svmlight_file(X, y, output_libsvm_path, zero_based=True)

# Convert training and testing data to LIBSVM format
csv_to_libsvm('train.csv', 'train_libsvm.txt', label_column)
csv_to_libsvm('test.csv', 'test_libsvm.txt', label_column)

# Upload converted data to S3
train_libsvm_s3_path = os.path.join(prefix, 'train/train_libsvm.txt')
s3.Bucket(bucket_name).Object(train_libsvm_s3_path).upload_file('train_libsvm.txt')

test_libsvm_s3_path = os.path.join(prefix, 'validation/test_libsvm.txt')
s3.Bucket(bucket_name).Object(test_libsvm_s3_path).upload_file('test_libsvm.txt')

print(f"Training data uploaded to S3: s3://{bucket_name}/{train_libsvm_s3_path}")
print(f"Testing data uploaded to S3: s3://{bucket_name}/{test_libsvm_s3_path}")


Training data uploaded to S3: s3://bankapplicationnew12/xgboost-as-a-built-in-algo/train/train_libsvm.txt
Testing data uploaded to S3: s3://bankapplicationnew12/xgboost-as-a-built-in-algo/validation/test_libsvm.txt


In [24]:
# Prepare S3 input for SageMaker
s3_input_train = TrainingInput(s3_data=f's3://{bucket_name}/{train_libsvm_s3_path}', content_type='libsvm')
s3_input_test = TrainingInput(s3_data=f's3://{bucket_name}/{test_libsvm_s3_path}', content_type='libsvm')
print("S3 input for SageMaker created successfully")

# Define container and hyperparameters
container = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=boto3.Session().region_name,
    version='1.0-1'
)
hyperparameters = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "binary:logistic",
    "num_round": "100"  # Add this line
}

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


S3 input for SageMaker created successfully


In [25]:
# Initialize and fit estimator
role = sagemaker.get_execution_role()
estimator = sagemaker.estimator.Estimator(
    image_uri=container,
    hyperparameters=hyperparameters,
    role=role,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    volume_size=5,
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600
)

estimator.fit({'train': s3_input_train, 'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-07-05-06-15-04-635


2024-07-05 06:15:04 Starting - Starting the training job...
2024-07-05 06:15:21 Starting - Preparing the instances for training...
2024-07-05 06:15:57 Downloading - Downloading the training image...
2024-07-05 06:16:33 Training - Training image download completed. Training in progress....
2024-07-05 06:16:53 Uploading - Uploading generated training model[2024-07-05 06:16:48.184 ip-10-0-194-137.eu-north-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Single node training.
[06:16:48] 28831x60 matrix with 434598 entries loaded from /opt/ml/input/data/train
[06:16:48] 12357x60 matrix with 18628